<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-cleaning" data-toc-modified-id="Data-cleaning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data cleaning</a></span><ul class="toc-item"><li><span><a href="#Merge-rows-with-same-book_id-and-movie_id." data-toc-modified-id="Merge-rows-with-same-book_id-and-movie_id.-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Merge rows with same <code>book_id</code> and <code>movie_id</code>.</a></span></li></ul></li><li><span><a href="#EDA" data-toc-modified-id="EDA-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>EDA</a></span><ul class="toc-item"><li><span><a href="#Books-explorations" data-toc-modified-id="Books-explorations-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Books explorations</a></span></li><li><span><a href="#Movies-EDA" data-toc-modified-id="Movies-EDA-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Movies EDA</a></span></li></ul></li></ul></div>

In [1]:
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import seaborn as sns
import requests

from collections import Counter

In [2]:
pd.__version__

'0.25.0'

In [3]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

%matplotlib inline
plt.style.use('fivethirtyeight')

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "last_expr"

%config InlineBackend.figure_format ='retina'

In [4]:
books = pd.DataFrame()
movies = pd.DataFrame()
books_s1 = pd.DataFrame()
movies_s1 = pd.DataFrame()

def read():
    global books
    global movies
    
    books = pd.read_csv('data/books.csv')
    movies = pd.read_csv('data/movies.csv')
    
read()

In [36]:
books.head()

,book_id,author,book_title,book_wiki_url,count,adaptations,isbn,oclc,valid_identifier,metadata_updated,number_of_pages,subject_places,subjects,publish_date,publish_places,goodreads,gc_title,gc_language_code,gc_average_rating,gc_format,gc_ratings_count,gc_text_reviews_count,title,gc_authors
0,b_9780385007511_848489,Stephen King,'Salem's Lot,https://en.wikipedia.org/wiki/%27Salem%27s_Lot,0,[],9780385007511,848489,True,True,427.0,"['Maine', 'U.S.A.', ""Jerusalem's Lot"", 'Marste...","['Catholic Church', 'arson', 'Great Depression...",1976,['New York'],1589604.0,Salem's Lot,en-US,4.01,Hardcover,443.0,65.0,'salem's Lot,NaN
1,b_0684819066_38377745,NaN,A Beautiful Mind_(book),https://en.wikipedia.org/wiki/A Beautiful Mind...,1,['A Beautiful Mind'],0684819066,38377745,True,True,459.0,['United States'],"['Mathematicians', 'Biography']",1998,['New York'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,A beautiful mind,NaN
2,b_0434098000_4205836,Anthony Burgess,A Clockwork Orange (novel),https://en.wikipedia.org/wiki/A_Clockwork_Oran...,0,[],0434098000,4205836,True,True,195.0,"['England', 'Korova Milk Bar']","['Internet Archive Wishlist', 'bible', 'aversi...",1962,['London'],101194.0,A Clockwork Orange,eng,3.99,Hardcover,142.0,17.0,A clockwork orange,NaN
3,b_0434098000_4205836,Anthony Burgess,A Clockwork Orange_(novel),https://en.wikipedia.org/wiki/A Clockwork Oran...,1,['A Clockwork Orange'],0434098000,4205836,True,True,195.0,"['England', 'Korova Milk Bar']","['Internet Archive Wishlist', 'bible', 'aversi...",1962,['London'],101194.0,A Clockwork Orange,eng,3.99,Hardcover,142.0,17.0,A clockwork orange,NaN
4,b_0064433390_30559641,William Joyce,A Day with Wilbur Robinson,https://en.wikipedia.org/wiki/A_Day_with_Wilbu...,0,[],0064433390,30559641,True,True,32.0,NaN,"['In library', 'Humorous stories']","September 30, 1993",NaN,566625.0,A Day with Wilbur Robinson,NaN,4.03,Paperback,647.0,81.0,A Day with Wilbur Robinson,NaN


In [ ]:
movies.head()

## Data cleaning

In [5]:
def lower():
    global books
    global movies
    
    books = books.applymap(lambda c:c.lower().strip() if type(c) == str else c)
    movies = movies.applymap(lambda c:c.lower().strip() if type(c) == str else c)
    
    nan_to_string = ['author', 'isbn', 'oclc', 'title', 'imdb_rated', 'imdb_runtime']

    for col_name in nan_to_string:
        try:
            books[col_name][books[col_name].isnull()] = ''
            movies[col_name][movies[col_name].isnull()] = ''
        except KeyError:
            pass

In [ ]:
books.shape, movies.shape

In [ ]:
msno.matrix(books.sample(250))

In [ ]:
msno.matrix(movies.sample(250))

In [ ]:
def get_dtype_summary(df):
    x = pd.DataFrame(df.dtypes, columns=['dtype']).rename_axis('keys')
    y = pd.DataFrame(df.isnull().sum(), columns= ['null_count']).rename_axis('keys')
    z = pd.DataFrame(df.count(), columns = ['non_null_count']).rename_axis('keys')
    x = x.merge(y.merge(z, on = 'keys'), on = 'keys')
    print(x)

In [ ]:
get_dtype_summary(books)

In [ ]:
get_dtype_summary(movies)

### Merge rows with same `book_id` and `movie_id`.

In [ ]:
sum(books['book_id'].value_counts() > 1), sum(
    movies['movie_id'].value_counts() > 1)

In [ ]:
books['book_id'].value_counts()[:7]
movies['movie_id'].value_counts()[:7]

In [ ]:
books.loc[books['book_id'] == 'b_9780307931894_972369283']

In [ ]:
movies.loc[movies['movie_id'] == 'm__53071567_tt0857480'][:4]

Row merge

In [ ]:
books = books[books['book_id'] != 'b_broken_broken']

In [6]:
subjects_to_exclude = ['protected daisy', 'fiction', 'in library', 'accessible book',
                       'fiction', 'internet archive wishlist', 'large type books', 'in library',
                       'popular print disabled books', 'open library staff picks', 'ficción',
                       'fiction in english', 'accessible book', 'ficción juvenil', 'open_syllabus_project',
                       'overdrive']

def remove_books_subs():
    global books_ex
    books_ex = books_ex[books_ex['subjects'].isin(subjects_to_exclude)]
    
    print('Books_ex', books_ex.shape)

In [7]:
def row_search(df, col, query, condensed = False):
    
    if condensed is False:zip
        return df[df[col] == query]
    else:
        if id(df) == id(books):
            book_id = df[df[col] == query]['book_id'][1]
            title = df[df[col] == query]['book_title'][1]
            author = df[df[col] == query]['author'][1]
            
            return {'book_id': book_id, 'title': title, 'author': author}
        else:
            movie_id = df[df[col] == query]['movie_id'][1]
            movie_title = df[df[col] == query]['movie_title'][1]
            book_title = df[df[col] == query]['book_title'][1]
            
            return {'movie_id': movie_id, 'movie_title': movie_title, 'book_title': book_title}

IndentationError: unexpected indent (<ipython-input-7-58add2be7592>, line 4)

In [ ]:
books.columns

In [ ]:
x = row_search(books, 'book_id', 'b_0684819066_38377745', True)
x

In [ ]:
def clean_dates():
    try:
        books['gc_text_reviews_count'] = books['gc_text_reviews_count'] + 1
        books['publish_date'] = pd.to_datetime(books['publish_date'])
        books['publish_year'] = books['publish_date'].dt.year
        
        movies['imdb_released'] = pd.to_datetime(movies['imdb_released'])
    except ValueError or AttributeError:
        pass

    try:
        books.drop(columns=['book_wiki_url', 'adaptations', 'title', 'gc_title'], inplace = True)
    except KeyError:
        pass

    print('Books:', books.shape, ', Movies:', movies.shape)

In [ ]:
books_ex = pd.DataFrame()
movies_ex = pd.DataFrame()
merge_ex = pd.DataFrame()

books_ex_save = pd.DataFrame()
movies_ex_save = pd.DataFrame()
merge_ex_save = pd.DataFrame()



In [ ]:
def save():
    global books_s1
    global movies_s1
    global books_ex_save
    global movies_ex_save
    global merge_ex_save
    
    books_s1 = books.copy(deep=True)
    movies_s1 = movies.copy(deep=True)
    
    books_ex_save = books_ex.copy(deep=True)
    movies_ex_save = movies_ex.copy(deep=True)
    merge_ex_save = merge_ex.copy(deep=True)
    

def restore():
    global books
    global movies
    global books_ex
    global movies_ex
    global merge_ex
    
    books = books_s1.copy(deep=True)
    movies = movies_s1.copy(deep=True)
    
    books_ex = books_ex_save.copy(deep=True)
    movies_ex = movies_ex_save.copy(deep=True)
    merge_ex = merge_ex_save.copy(deep=True)
    
    

In [ ]:
def col_clean(df, list_of_columns):
    for col in list_of_columns:
        df[col] = df[col].apply(list_clean, convert_dtype = True)
    print(df.shape)

def list_clean(str_list):
    str_list = str(str_list)
    cleaned_list = str_list.replace("'", '').replace('[', '').replace(']', '').replace('"', '')
    cleaned_list = [element.strip() for element in cleaned_list.split(',')]
    
    if len(cleaned_list) == 1:
        if 'nan' in cleaned_list:
            cleaned_list = None
        else:
            cleaned_list = cleaned_list[0]
    
    return cleaned_list

list_clean("['secret service', 'world war, 1939-1945', 'fiction']")

In [ ]:
read()
lower()
books = books.groupby(['book_id']).agg('max').reset_index()
movies = movies.groupby(['movie_id']).agg('max').reset_index()
col_clean(books, ['subjects', 'subject_places', 'publish_places'])
col_clean(movies, ['imdb_actors', 'imdb_writers', 'imdb_genres'])
clean_dates()


In [41]:
books.head(5)

,book_id,author,book_title,count,isbn,oclc,valid_identifier,metadata_updated,number_of_pages,subject_places,subjects,publish_date,publish_places,goodreads,gc_language_code,gc_average_rating,gc_format,gc_ratings_count,gc_text_reviews_count,gc_authors,publish_year
0,b_0002212080_1993343,jack higgins,the eagle has landed (novel),0,0002212080,1993343,True,True,361.0,germany,"[secret service, world war, 1939-1945, fiction]",1976-01-01,toronto,1181359.0,eng,4.16,mass market paperback,94.0,9.0,NaN,1976.0
1,b_0002227118_31704568,patrick o'brian,the far side of the world,1,0002227118,31704568,True,True,371.0,"[great britain, oceania, united states]","[british naval operations, fiction, history, j...",1984-01-01,london,2771584.0,eng,4.45,hardcover,3.0,1.0,NaN,1984.0
2,b_0002239728_60115623,robert ludlum,the apocalypse watch,0,0002239728,60115623,True,True,645.0,"[united states, europe]","[fiction, brothers, conspiracies, intelligence...",1995-01-01,london,275759.0,NaN,3.88,NaN,6.0,1.0,NaN,1995.0
3,b_0002240262_30814041,,ring_(book),1,0002240262,30814041,True,True,336.0,None,None,1994-07-12,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994.0
4,b_0006158056_16481828,alistair maclean,breakheart pass (novel),0,0006158056,16481828,True,True,192.0,None,"[in library, fiction in english, protected daisy]",1980-01-01,(london),149607.0,eng,3.66,paperback,2123.0,59.0,NaN,1980.0


In [ ]:
books_ex = books.explode('subject_places').explode('subjects')
movies_ex = movies.explode('imdb_genres').explode('imdb_actors').explode('imdb_writers')

books_ex.drop_duplicates(subset = ['subject_places', 'subjects'], inplace = True)
movies_ex.drop_duplicates(subset = ['imdb_actors', 'imdb_genres', 'imdb_writers'], inplace = True)

remove_books_subs()
save()

books_ex.shape, movies_ex.shape

In [45]:
books.loc[books['author'] == '']

,book_id,author,book_title,count,isbn,oclc,valid_identifier,metadata_updated,number_of_pages,subject_places,subjects,publish_date,publish_places,goodreads,gc_language_code,gc_average_rating,gc_format,gc_ratings_count,gc_text_reviews_count,gc_authors,publish_year
3,b_0002240262_30814041,,ring_(book),1,0002240262,30814041,True,True,336.0,None,None,1994-07-12,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994.0
13,b_0060817089_58431841,,marley & me,1,0060817089,58431841,True,True,NaN,florida,"[anecdotes, labrador retriever, puppies, human...",2005-10-18,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.0
18,b_0062024027_769412945,,divergent_(novel),1,0062024027,769412945,True,True,NaN,chicago,"[new york times bestseller, nyt:paperback_book...",2011-01-01,new york,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.0
19,b_0062234935_794210765,,insurgent_(novel),1,0062234935,794210765,True,False,NaN,None,None,NaT,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,b_0312930194_16268967,,seventh son_(novel),1,0312930194,16268967,True,False,NaN,None,None,NaT,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,b_0374299218_66463617,,call me by your name_(novel),1,0374299218,66463617,True,True,248.0,italy,"[teenage boys, accessible book, authors, in li...",2007-01-01,new york,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007.0
189,b_0399116974_1975222,,children of dune,1,0399116974,1975222,True,True,444.0,None,"[dune (imaginary place), fiction, fiction in e...",1976-01-01,new york,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1976.0
296,b_067940211_25632732,,primal fear_(novel),1,067940211,25632732,True,False,NaN,None,None,NaT,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
318,b_0684819066_38377745,,a beautiful mind_(book),1,0684819066,38377745,True,True,459.0,united states,"[mathematicians, biography]",1998-01-01,new york,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1998.0
344,b_074754624_,,harry potter and the goblet of fire,1,074754624,,True,False,NaN,None,None,NaT,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
x = movies.loc[movies['movie_title'] == 'oliver & company']
x['imdb_actors'][395]

In [ ]:
movies_ex['imdb_genres'].value_counts(normalize = True)

In [ ]:
movies_ex.loc[movies_ex['movie_id'] == 'm__185812519_tt0095776']

## EDA

### Books explorations

In [ ]:
books.columns

In [31]:
books['author'].value_counts()[1:25].plot(kind = 'bar', figsize = (15, 8))
np.mean(books['author'].value_counts())

KeyError: 'author'

In [ ]:
books['number_of_pages'].plot.hist(bins = 25, figsize = (10, 6))
np.nanmean(books['number_of_pages'])
np.nanstd([books['number_of_pages']])

In [ ]:
books['publish_year'].plot.hist(bins = 50, figsize = (10, 6))
np.nanmean(books['publish_year'])
np.nanstd(books['publish_year'])

In [ ]:
books['gc_language_code'].value_counts()
books['gc_format'].value_counts()

In [ ]:
#TODO: find average good reads book rating

In [ ]:
books['gc_average_rating'].plot.hist(bins = 20, figsize = (10, 5), xlim = [0, 5])
np.nanmean(books['gc_average_rating'])
np.nanstd(books['gc_average_rating'])

In [ ]:
mean_rating = np.mean(books['gc_average_rating'])

plt.figure(figsize=(10, 5))
plt.scatter(x=books['publish_year'],
            y=books['gc_average_rating'],  alpha=0.5, c='blue')
plt.ylim(2, 5.5)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(x=books['publish_year'],
            y=np.log(books['gc_text_reviews_count']),  alpha=0.5, c='blue')
plt.ylabel('Log of text reviews count')
plt.axhline(y = np.log(np.nanmean(books['gc_text_reviews_count'])), c = 'red')
plt.axhline(y = np.log(np.nanmedian(books['gc_text_reviews_count'])), c = 'green')
plt.title('Relationaship between log(text reviews) and publish year')
plt.show()


it seems that only the recent books are tagged, but there doesn't seem to be any sort of upwards trends in book ratings. --> people enjoy older books as much as newer books

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(x=books['gc_average_rating'],
            y=np.log(books['gc_text_reviews_count']),  alpha=0.5, c='blue')
plt.axhline(y = np.nanmedian(books['gc_average_rating']), c = 'green')
plt.ylabel('Log of text reviews count')
plt.xlim(2, 5)
plt.show()

In [ ]:
books.columns

In [ ]:
sample = books.iloc[343]
sample
sample.subjects
sample.subject_places

In [ ]:
books_subjects = []
subject_places = []
publish_places = []
books_subjects_count = None

subjects_to_exclude = ['protected daisy', 'fiction', 'in library', 'accessible book',
                       'fiction', 'internet archive wishlist', 'large type books', 'in library',
                       'popular print disabled books', 'open library staff picks', 'ficción',
                       'fiction in english', 'accessible book', 'ficción juvenil', 'open_syllabus_project',
                       'overdrive']

for index in books[['publish_year', 'subjects', 'subject_places', 'publish_places']].itertuples():
    subjects = index.subjects
    if subjects is not None and isinstance(subjects, str):
        x = subjects.replace("'", '').replace('[', '').replace(']', '').replace('"', '')
        x = x.split(',')
        books_subjects += [subject.strip()
                           for subject in x if subject.strip() not in subjects_to_exclude]
        
    s_places = index.subject_places
    if s_places is not None and isinstance(s_places, str):
        x = s_places.replace("'", '').replace('[', '').replace(']', '').replace('"', '').replace('u.s.a.', 'usa')
        x = x.split(',')
        subject_places += [place.strip() for place in x]
        
    p_places = index.publish_places
    if p_places is not None and isinstance(p_places, str):
        x = p_places.replace("'", '').replace('[', '').replace(']', '').replace('"', '').replace('u.s.a.', 'usa')
        x = x.split(',')
        publish_places += [place.strip() for place in x]

books_subjects_count = Counter(books_subjects)
subject_places_count = Counter(subject_places)
publish_places_count = Counter(publish_places)

books_subjects_count.most_common(20)
subject_places_count.most_common(20)
publish_places_count.most_common(20)

### Movies EDA

In [ ]:
movies.columns

In [ ]:
x = pd.DataFrame(data=movies['imdb_year'].value_counts())
x['year'] = x.index
x = x[pd.to_numeric(x['year'], errors='coerce').notnull()]
x.set_index('year', inplace = True)
x.sort_index(inplace = True)
x.rename(columns = {'imdb_year': 'count'})
x.columns

In [ ]:
print('ha')